In [1]:
import pandas as pd
import nltk
import re
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("tweet_emotions2.csv", encoding='latin-1')
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
74787,surprise,@MichelGW have you gift! Hope you like it! It'...
74788,joy,The world didnt give it to me..so the world MO...
74789,anger,A man robbed me today .
74790,fear,"Youu call it JEALOUSY, I call it of #Losing YO..."


In [3]:
df.sentiment.value_counts()

sadness       11887
joy           11045
neutral       10892
worry          8459
surprise       6249
fear           5410
happiness      5209
anger          4407
love           3842
fun            1776
relief         1526
hate           1323
disgust         856
empty           827
enthusiasm      759
boredom         179
shame           146
Name: sentiment, dtype: int64

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def clean_tokenized_lemmatized(tweet):
    tweet = tweet.lower()                                                       #converting the text into lower
    tweet = re.sub(r'@\w+', '', tweet)                                          #removing @mentions
    tweet = re.sub(r'#\w+', '', tweet)                                          #removing hashtags
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)   #removing url          
    tweet = re.sub(r'[^\w\s]','',tweet)                                         #removing punctuations
    tweet = re.sub(r'\d+', '', tweet)                                           #removing numbers
    tweet = re.sub(r'\s+', ' ', tweet).strip()                                  #removing extra whitespaces  
    
    lemmatizer = WordNetLemmatizer()
    stops = stopwords.words('english')
    
    tokens = nltk.word_tokenize(tweet)
    tokens = [t for t in tokens if not t in stops]
    fintokens = []
    for token in tokens:
        fintokens.append(lemmatizer.lemmatize(token))
    finaltext = " "
    return finaltext.join(fintokens)

for i in df.index:
    string = df['content'][i]
    preprostr = clean_tokenized_lemmatized(string)
    df.at[i, 'content'] = preprostr


In [4]:
positive = ['joy', 'surprise', 'happiness', 'love', 'fun', 'relief', 'enthusiasm']
negative = ['sadness', 'worry', 'fear', 'anger', 'hate', 'disgust', 'boredom', 'shame']
neutral = ['neutral', 'empty']
def emotion_detection(value):
    if value in positive:
        return 1
    elif value in negative:
        return 0
    else:
        return 2
df['emotion'] = df['sentiment'].map(emotion_detection)

In [5]:
df

,sentiment,content,emotion
0,empty,@tiffanylue i know i was listenin to bad habi...,2
1,sadness,Layin n bed with a headache ughhhh...waitin o...,0
2,sadness,Funeral ceremony...gloomy friday...,0
3,enthusiasm,wants to hang out with friends SOON!,1
4,neutral,@dannycastillo We want to trade with someone w...,2
...,...,...,...
74787,surprise,@MichelGW have you gift! Hope you like it! It'...,1
74788,joy,The world didnt give it to me..so the world MO...,1
74789,anger,A man robbed me today .,0
74790,fear,"Youu call it JEALOUSY, I call it of #Losing YO...",0


In [6]:
df.emotion.value_counts()

0    32667
1    30406
2    11719
Name: emotion, dtype: int64

In [9]:
df

,sentiment,content,emotion
0,empty,know listenin bad habit earlier started freaki...,2
1,sadness,layin n bed headache ughhhhwaitin call,0
2,sadness,funeral ceremonygloomy friday,0
3,enthusiasm,want hang friend soon,1
4,neutral,want trade someone houston ticket one,2
...,...,...,...
74787,surprise,gift hope like hand made wear itll keep warm lol,1
74788,joy,world didnt give meso world definitely cnt tak...,1
74789,anger,man robbed today,0
74790,fear,youu call jealousy call,0


In [6]:
le_sentiment = LabelEncoder()

In [7]:
df['sentiment_n'] = le_sentiment.fit_transform(df['sentiment'])

In [8]:
df

,sentiment,content,emotion,sentiment_n
0,empty,know listenin bad habit earlier started freaki...,2,3
1,sadness,layin n bed headache ughhhhwaitin call,0,13
2,sadness,funeral ceremonygloomy friday,0,13
3,enthusiasm,want hang friend soon,1,4
4,neutral,want trade someone houston ticket one,2,11
...,...,...,...,...
74787,surprise,gift hope like hand made wear itll keep warm lol,1,15
74788,joy,world didnt give meso world definitely cnt tak...,1,9
74789,anger,man robbed today,0,0
74790,fear,youu call jealousy call,0,5


In [31]:
df.sentiment_n.value_counts()

13    11887
9     11045
11    10892
16     8459
15     6249
5      5410
7      5209
0      4407
10     3842
6      1776
12     1526
8      1323
2       856
3       827
4       759
1       179
14      146
Name: sentiment_n, dtype: int64

In [9]:
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(
    df.content, 
    df.sentiment_n,
    df.emotion,
    test_size=0.1, # 10% samples will go to test dataset
    random_state=2022
)

In [15]:
print(X_train.shape)
X_train.head()

(67312,)


72340        got ur pressie already bt u wont disappointed
66741    caaauuuteahh forgot made bed morning aww thank...
39901                                                 love
40288    hate train ran one missed second saw drive awa...
57798                                                  yes
Name: content, dtype: object

In [16]:
z_test.value_counts()


0    3281
1    3032
2    1167
Name: emotion, dtype: int64

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier

In [22]:
clf = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 3))),
     ('SVM', LinearSVC())         
])

In [23]:
clf.fit(X_train, z_train)

C:\Anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('SVM', LinearSVC())])

In [25]:
z_pred = clf.predict(X_test)

In [26]:
print(classification_report(z_test, z_pred))

              precision    recall  f1-score   support

           0       0.73      0.70      0.71      3281
           1       0.68      0.66      0.67      3032
           2       0.40      0.48      0.44      1167

    accuracy                           0.65      7480
   macro avg       0.60      0.61      0.61      7480
weighted avg       0.66      0.65      0.65      7480



In [27]:
clf.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('SVM', LinearSVC())])

In [28]:
y_pred = clf.predict(X_test)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.50      0.54       418
           1       0.00      0.00      0.00        12
           2       0.65      0.17      0.27        78
           3       0.05      0.02      0.03        97
           4       0.12      0.03      0.04        74
           5       0.77      0.69      0.73       556
           6       0.19      0.06      0.09       190
           7       0.27      0.28      0.27       484
           8       0.26      0.15      0.19       157
           9       0.54      0.58      0.56      1110
          10       0.39      0.34      0.36       393
          11       0.32      0.59      0.42      1070
          12       0.09      0.04      0.05       152
          13       0.43      0.43      0.43      1182
          14       0.75      0.86      0.80        14
          15       0.43      0.30      0.35       629
          16       0.26      0.22      0.24       864

    accuracy              

In [11]:
clf2 = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 3))),
     ('DT', DecisionTreeClassifier())         
])

In [12]:
clf2.fit(X_train, z_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('DT', DecisionTreeClassifier())])

In [13]:
z_pred = clf2.predict(X_test)

In [14]:
print(classification_report(z_test, z_pred))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66      3281
           1       0.63      0.57      0.60      3032
           2       0.34      0.45      0.39      1167

    accuracy                           0.59      7480
   macro avg       0.55      0.56      0.55      7480
weighted avg       0.60      0.59      0.59      7480



In [15]:
clf2.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('DT', DecisionTreeClassifier())])

In [16]:
y_pred = clf2.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.42      0.45       418
           1       0.00      0.00      0.00        12
           2       0.30      0.13      0.18        78
           3       0.02      0.01      0.01        97
           4       0.02      0.01      0.02        74
           5       0.73      0.61      0.66       556
           6       0.07      0.03      0.04       190
           7       0.20      0.18      0.19       484
           8       0.25      0.18      0.21       157
           9       0.44      0.44      0.44      1110
          10       0.28      0.27      0.28       393
          11       0.29      0.53      0.37      1070
          12       0.07      0.04      0.05       152
          13       0.36      0.35      0.36      1182
          14       0.86      0.86      0.86        14
          15       0.32      0.21      0.25       629
          16       0.23      0.20      0.21       864

    accuracy              

In [18]:
clf3 = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 3))),
     ('XGB', XGBClassifier())         
])

In [19]:
clf3.fit(X_train, z_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('XGB',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=Non

In [20]:
z_pred = clf3.predict(X_test)

In [21]:
print(classification_report(z_test, z_pred))

              precision    recall  f1-score   support

           0       0.57      0.83      0.68      3281
           1       0.69      0.61      0.65      3032
           2       0.56      0.04      0.07      1167

    accuracy                           0.62      7480
   macro avg       0.61      0.49      0.46      7480
weighted avg       0.62      0.62      0.57      7480



In [22]:
clf3.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('XGB',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=Non

In [47]:
y_pred = clf3.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.42      0.45       418
           1       0.00      0.00      0.00        12
           2       0.30      0.13      0.18        78
           3       0.02      0.01      0.01        97
           4       0.02      0.01      0.02        74
           5       0.73      0.61      0.66       556
           6       0.07      0.03      0.04       190
           7       0.20      0.18      0.19       484
           8       0.25      0.18      0.21       157
           9       0.44      0.44      0.44      1110
          10       0.28      0.27      0.28       393
          11       0.29      0.53      0.37      1070
          12       0.07      0.04      0.05       152
          13       0.36      0.35      0.36      1182
          14       0.86      0.86      0.86        14
          15       0.32      0.21      0.25       629
          16       0.23      0.20      0.21       864

    accuracy              

In [25]:
clf4 = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 3))),
     ('NB', MultinomialNB())         
])

In [26]:
clf4.fit(X_train, z_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('NB', MultinomialNB())])

In [27]:
z_pred = clf4.predict(X_test)

In [28]:
print(classification_report(z_test, z_pred))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      3281
           1       0.65      0.71      0.68      3032
           2       0.58      0.01      0.02      1167

    accuracy                           0.64      7480
   macro avg       0.62      0.51      0.47      7480
weighted avg       0.63      0.64      0.59      7480



In [29]:
clf4.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 3))),
                ('NB', MultinomialNB())])

In [32]:
y_pred = clf4.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.33      0.49       418
           1       0.00      0.00      0.00        12
           2       1.00      0.06      0.12        78
           3       0.00      0.00      0.00        97
           4       0.00      0.00      0.00        74
           5       0.88      0.59      0.70       556
           6       0.00      0.00      0.00       190
           7       0.31      0.12      0.17       484
           8       0.00      0.00      0.00       157
           9       0.40      0.74      0.51      1110
          10       0.64      0.18      0.29       393
          11       0.36      0.23      0.28      1070
          12       0.00      0.00      0.00       152
          13       0.27      0.72      0.39      1182
          14       0.00      0.00      0.00        14
          15       0.64      0.15      0.25       629
          16       0.28      0.20      0.23       864

    accuracy              

C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
